In [1]:
# === Imports ===
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# === Parameters ===
DATA_PATH = "/home/renga/Desktop/neoen_data/renga_work/data/grid_penalty/combined_with_imbalance.csv"  
TIME_COL = "measure_date"
TARGET_COL = "imbalance"
DA_PRICE_COL = "da_price_15min"                        # optional exogenous

# Model horizons & cadence
HORIZON_HOURS  = 48   # <- change to alter forecast window (e.g., 24, 72, 96)
STEP_MINUTES   = 15   # data granularity
STRIDE_MINUTES = 60   # stride between rolling forecast origins during evaluation

# History window to build lag/rolling features
HISTORY_HOURS = 96    # <- change to 48/96/etc. to trade speed vs accuracy

# Feature toggles
USE_EXOG_DA = False   # <- set True to include lagged day-ahead price as exogenous

# Train/test split cutoff (inclusive for train) — UTC as required
TRAIN_CUTOFF_UTC = "2025-08-15 23:59:59+00:00"

# Optional strict 15-min reindexing. Set to None to skip.
FREQ = "15T"

# Evaluation speed control (set to None for full coverage; can be slower)
MAX_ORIGINS = 200

RANDOM_STATE = 42




In [4]:
# If running locally and missing dependencies, uncomment:
# %pip install -q pandas numpy scikit-learn plotly ipywidgets
# %pip install -q pyarrow  # faster CSV IO (optional)


In [3]:
# Load data
df = pd.read_csv(DATA_PATH)
assert TIME_COL in df.columns, f"Missing '{TIME_COL}' in columns: {df.columns.tolist()}"
assert TARGET_COL in df.columns, f"Missing '{TARGET_COL}' in columns: {df.columns.tolist()}"

# Parse datetime & sort
df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=True, errors="coerce")
df = df.dropna(subset=[TIME_COL]).sort_values(TIME_COL).reset_index(drop=True)

# Optional: align to strict 15-min grid (reindex)
if FREQ is not None:
    start, end = df[TIME_COL].min(), df[TIME_COL].max()
    full_index = pd.date_range(start=start, end=end, freq=FREQ, tz="UTC")
    df = df.set_index(TIME_COL).reindex(full_index)
    df.index.name = TIME_COL
    df = df.reset_index()

print("Shape:", df.shape)
print("Columns:", list(df.columns)[:20])
print("Date range:", df[TIME_COL].min(), "->", df[TIME_COL].max())
df.head(3)


Shape: (8545, 6)
Columns: ['measure_date', 'measure_date_UTC', 'imbalance', 'positive_imbalance_settlement_price', 'negative_imbalance_settlement_price', 'da_price_15min']
Date range: 2025-07-06 22:00:00+00:00 -> 2025-10-03 22:00:00+00:00


,measure_date,measure_date_UTC,imbalance,positive_imbalance_settlement_price,negative_imbalance_settlement_price,da_price_15min
0,2025-07-06 22:00:00+00:00,2025-07-06 22:00:00+00:00,-128.0,42.61,46.15,46.35
1,2025-07-06 22:15:00+00:00,2025-07-06 22:15:00+00:00,-104.0,39.36,42.62,46.35
2,2025-07-06 22:30:00+00:00,2025-07-06 22:30:00+00:00,27.0,-30.10,-27.80,46.35


In [4]:
ts_obj = pd.Timestamp(TRAIN_CUTOFF_UTC)
TRAIN_CUTOFF_UTC = ts_obj if ts_obj.tzinfo else ts_obj.tz_localize("UTC")

train_df = df[df[TIME_COL] <= TRAIN_CUTOFF_UTC].copy()
test_df  = df[df[TIME_COL] >  TRAIN_CUTOFF_UTC].copy()

print("Train period:", train_df[TIME_COL].min(), "->", train_df[TIME_COL].max(), "rows:", len(train_df))
print("Test  period:",  test_df[TIME_COL].min(),  "->", test_df[TIME_COL].max(),  "rows:", len(test_df))
print("NaNs in target (train/test):", train_df[TARGET_COL].isna().sum(), "/", test_df[TARGET_COL].isna().sum())


Train period: 2025-07-06 22:00:00+00:00 -> 2025-08-15 23:45:00+00:00 rows: 3848
Test  period: 2025-08-16 00:00:00+00:00 -> 2025-10-03 22:00:00+00:00 rows: 4697
NaNs in target (train/test): 0 / 0


In [5]:
def build_features(
    df_in: pd.DataFrame,
    time_col: str,
    target_col: str,
    n_lags: int,
    roll_windows: List[int],
    use_exog_da: bool = False,
    da_col: str = "da_price_15min",
    add_time_feats: bool = False,
) -> pd.DataFrame:
    """
    Convert a time series into a supervised table for 1-step-ahead prediction.
    Uses only past information (lags & rolling stats). Optionally include lagged DA price.
    Optionally add time-of-day / day-of-week cyclical features when add_time_feats=True.
    """
    df = df_in.copy().set_index(time_col).sort_index()
    y = df[target_col].astype(float).copy()

    feat = pd.DataFrame(index=df.index)

    # Optional time features (hour of day, minute, day-of-week) and cyclical encoding
    if add_time_feats:
        ts = df.index
        feat["hour"] = ts.hour
        feat["minute"] = ts.minute
        feat["dow"] = ts.dayofweek
        # cyclical encoding of time-of-day (better for circular patterns)
        seconds_in_day = 24 * 3600
        tod_seconds = ts.hour * 3600 + ts.minute * 60 + ts.second
        feat["tod_sin"] = np.sin(2 * np.pi * tod_seconds / seconds_in_day)
        feat["tod_cos"] = np.cos(2 * np.pi * tod_seconds / seconds_in_day)

    # Lags of target
    for lag in range(1, n_lags + 1):
        feat[f"lag_{lag}"] = y.shift(lag)

    # Rolling stats on target (shifted to avoid peeking)
    for w in roll_windows:
        feat[f"roll_mean_{w}"] = y.shift(1).rolling(w, min_periods=max(2, w//2)).mean()
        feat[f"roll_std_{w}"]  = y.shift(1).rolling(w, min_periods=max(2, w//2)).std()

    # Optional exogenous: lagged day-ahead price (safe to include if known before delivery)
    if use_exog_da and (da_col in df.columns):
        da = df[da_col].astype(float).copy()
        for lag in range(0, min(96, n_lags+8)):  # ~24h context
            feat[f"da_lag_{lag}"] = da.shift(lag)
        for w in roll_windows:
            feat[f"da_roll_mean_{w}"] = da.shift(1).rolling(w, min_periods=max(2, w//2)).mean()

    # Target for 1-step ahead
    feat["y_next"] = y.shift(-1)

    feat = feat.dropna().reset_index().rename(columns={time_col: "ts"})
    return feat

# Derive steps from cadence
steps_per_hour = 60 // STEP_MINUTES
N_LAGS = HISTORY_HOURS * steps_per_hour
ROLL_WINDOWS = [2*steps_per_hour, 6*steps_per_hour, 12*steps_per_hour, 24*steps_per_hour]

print("Derived: steps/hour=", steps_per_hour, "| N_LAGS=", N_LAGS, "| roll windows=", ROLL_WINDOWS)


Derived: steps/hour= 4 | N_LAGS= 384 | roll windows= [8, 24, 48, 96]


In [6]:
needed_cols = [TIME_COL, TARGET_COL] + ([DA_PRICE_COL] if USE_EXOG_DA else [])
train_feat = build_features(
    train_df[needed_cols], time_col=TIME_COL, target_col=TARGET_COL,
    n_lags=N_LAGS, roll_windows=ROLL_WINDOWS, use_exog_da=USE_EXOG_DA, da_col=DA_PRICE_COL
)
test_feat = build_features(
    test_df[needed_cols], time_col=TIME_COL, target_col=TARGET_COL,
    n_lags=N_LAGS, roll_windows=ROLL_WINDOWS, use_exog_da=USE_EXOG_DA, da_col=DA_PRICE_COL
)

feature_cols = [c for c in train_feat.columns if c not in ("ts","y_next")]
X_train = train_feat[feature_cols].values
y_train = train_feat["y_next"].values
X_test  = test_feat[feature_cols].values
y_test  = test_feat["y_next"].values

model = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("reg", Ridge(alpha=1.0, random_state=RANDOM_STATE))
])
model.fit(X_train, y_train)

y_pred_1 = model.predict(X_test)
mae_1 = mean_absolute_error(y_test, y_pred_1)
# rmse_1 = mean_squared_error(y_test, y_pred_1, squared=False)
print(f"1-step ahead Test MAE:  {mae_1:.2f}")
# print(f"1-step ahead Test RMSE: {rmse_1:.2f}")
print("Train supervised shape:", X_train.shape, "| Test supervised shape:", X_test.shape)
def rolling_forecast_evaluation(
    df_in: pd.DataFrame,
    time_col: str,
    target_col: str,
    model: Pipeline,
    horizon_steps: int,
    step_minutes: int,
    stride_minutes: int,
    n_lags: int,
    roll_windows: List[int],
    use_exog_da: bool = False,
    da_col: str = "da_price_15min",
    max_origins: int = None,
) -> pd.DataFrame:
    """
    Perform rolling forecast evaluation over multiple origins.
    Returns a DataFrame with forecasts and actuals for each horizon step.
    """
    df = df_in.copy().set_index(time_col).sort_index()
    results = []

    total_minutes = (df.index[-1] - df.index[0]).total_seconds() / 60
    total_steps = int(total_minutes // step_minutes)
    stride_steps = stride_minutes // step_minutes

    origin_indices = range(n_lags, total_steps - horizon_steps, stride_steps)
    if max_origins is not None:
        origin_indices = list(origin_indices)[:max_origins]

    for origin_step in origin_indices:
        origin_time = df.index[origin_step]
        train_window = df.iloc[:origin_step]

        feat = build_features(
            train_window.reset_index(),
            time_col=time_col,
            target_col=target_col,
            n_lags=n_lags,
            roll_windows=roll_windows,
            use_exog_da=use_exog_da,
            da_col=da_col
        )

        if feat.empty:
            continue

        X_origin = feat[feature_cols].values[-1].reshape(1, -1)

        preds = []
        for h in range(horizon_steps):
            y_pred_h = model.predict(X_origin)[0]
            preds.append(y_pred_h)

            # Update features for next step
            new_row = {f"lag_{i+1}": X_origin[0][i] for i in range(n_lags-1)}
            new_row["lag_1"] = y_pred_h

            for w in roll_windows:
                lag_indices = [i for i in range(n_lags) if i < w]
                lag_values = [X_origin[0][i] for i in lag_indices]
                new_row[f"roll_mean_{w}"] = np.mean(lag_values) if lag_values else np

/tmp/ipykernel_17011/4147035042.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat[f"lag_{lag}"] = y.shift(lag)
/tmp/ipykernel_17011/4147035042.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat[f"lag_{lag}"] = y.shift(lag)
/tmp/ipykernel_17011/4147035042.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

1-step ahead Test MAE:  129.85
Train supervised shape: (3463, 392) | Test supervised shape: (4312, 392)


In [7]:
# Feature order for recursive forecasting must match training's feature_cols
FEAT_ORDER = feature_cols.copy()

def xrow_from_buffer(buf: np.ndarray, n_lags: int, roll_windows: List[int]) -> np.ndarray:
    """Build a single feature row in FEAT_ORDER from the history buffer."""
    parts = {}
    # Lags: lag_1..lag_n (most recent first)
    lags_vals = buf[-n_lags:][::-1]
    for i, v in enumerate(lags_vals, start=1):
        parts[f"lag_{i}"] = v
    # Rolling stats (exclude the most recent point)
    for w in roll_windows:
        if len(buf) < w + 1:
            use = buf[:-1] if len(buf) > 1 else buf
        else:
            use = buf[-(w+1):-1]
        parts[f"roll_mean_{w}"] = float(np.mean(use)) if len(use) else float(buf[-1])
        parts[f"roll_std_{w}"]  = float(np.std(use, ddof=1)) if len(use) > 1 else 0.0

    # If DA exogenous was enabled, fill any missing da_* keys with zeros to match shape
    if USE_EXOG_DA:
        for k in FEAT_ORDER:
            if k not in parts:
                parts[k] = 0.0

    # Order features
    return np.array([parts[k] for k in FEAT_ORDER], dtype=float).reshape(1, -1)

def recursive_forecast_fast(history_values: np.ndarray, H: int, n_lags: int, roll_windows: List[int]) -> np.ndarray:
    """Recursive forecast using the trained 1-step model."""
    buf = np.asarray(history_values, dtype=float).copy()
    preds = np.empty(H, dtype=float)
    for t in range(H):
        x_row = xrow_from_buffer(buf, n_lags=n_lags, roll_windows=roll_windows)
        preds[t] = float(model.predict(x_row)[0])
        buf = np.append(buf, preds[t])
    return preds

# Build continuous series across train+test for history access
full = pd.concat([train_df[[TIME_COL, TARGET_COL]], test_df[[TIME_COL, TARGET_COL]]]).sort_values(TIME_COL).reset_index(drop=True)
full = full.dropna(subset=[TARGET_COL])
full_idx = pd.Index(full[TIME_COL])

H = int(HORIZON_HOURS * (60 // STEP_MINUTES))
stride = max(1, STRIDE_MINUTES // STEP_MINUTES)

# Rolling origins within the test region
origins = []
start_origin = test_df[TIME_COL].min()
for i in range(0, len(test_df), stride):
    ts = test_df.iloc[i][TIME_COL]
    if ts >= start_origin:
        origins.append(ts)
if MAX_ORIGINS is not None:
    origins = origins[:MAX_ORIGINS]

rows = []
y_map = dict(zip(full[TIME_COL].astype(np.int64), full[TARGET_COL]))

for origin in origins:
    pos = full_idx.get_loc(origin)
    if isinstance(pos, slice):
        pos = pos.start
    if pos < N_LAGS:
        continue  # not enough history

    history = full.iloc[:pos][TARGET_COL].values[-N_LAGS:]
    preds = recursive_forecast_fast(history_values=history, H=H, n_lags=N_LAGS, roll_windows=ROLL_WINDOWS)

    for h in range(1, H+1):
        target_ts = origin + pd.Timedelta(minutes=STEP_MINUTES*h)
        key = int(target_ts.value)
        y_true = y_map.get(key, np.nan)
        rows.append({
            "origin_ts": origin,
            "h": h,
            "target_ts": target_ts,
            "y_pred": preds[h-1],
            "y_true": y_true
        })

rolling_forecasts = pd.DataFrame(rows)
valid = rolling_forecasts.dropna(subset=["y_true"])
metrics = valid.groupby("h").apply(lambda g: pd.Series({
    "MAE":  mean_absolute_error(g["y_true"], g["y_pred"]),
    # "RMSE": mean_squared_error(g["y_true"], g["y_pred"], squared=False),
    "Count": len(g)
})).reset_index()

print("Origins evaluated:", len(origins), "| Horizon steps:", H, "| Rows:", len(rolling_forecasts))
metrics.head()


Origins evaluated: 200 | Horizon steps: 192 | Rows: 38400


,h,MAE,Count
0,1,101.033855,200.0
1,2,106.582757,200.0
2,3,136.022746,200.0
3,4,142.248230,200.0
4,5,149.671331,200.0


In [8]:
# Quick experiment: train & evaluate a model including time-of-day / day-of-week features on a small subset
Q_N_LAGS = min(N_LAGS, 96)  # use smaller lag for a fast check
N_TRAIN = 5000
N_TEST = 1000
train_sub = train_df.tail(N_TRAIN) if len(train_df) > N_TRAIN else train_df
test_sub  = test_df.head(N_TEST) if len(test_df) > N_TEST else test_df
needed_cols = [TIME_COL, TARGET_COL] + ([DA_PRICE_COL] if USE_EXOG_DA else [])
train_feat_tf = build_features(train_sub[needed_cols], time_col=TIME_COL, target_col=TARGET_COL, n_lags=Q_N_LAGS, roll_windows=ROLL_WINDOWS, use_exog_da=USE_EXOG_DA, da_col=DA_PRICE_COL, add_time_feats=True)
test_feat_tf  = build_features(test_sub[needed_cols],  time_col=TIME_COL, target_col=TARGET_COL, n_lags=Q_N_LAGS, roll_windows=ROLL_WINDOWS, use_exog_da=USE_EXOG_DA, da_col=DA_PRICE_COL, add_time_feats=True)
if train_feat_tf.empty or test_feat_tf.empty:
    print('Not enough rows to build time-feature test with Q_N_LAGS=', Q_N_LAGS)
else:
    feat_tf_cols = [c for c in train_feat_tf.columns if c not in ('ts','y_next')]
    X_train_tf = train_feat_tf[feat_tf_cols].values
    y_train_tf = train_feat_tf['y_next'].values
    X_test_tf  = test_feat_tf[feat_tf_cols].values
    y_test_tf  = test_feat_tf['y_next'].values
    model_tf = Pipeline([('scaler', StandardScaler()), ('reg', Ridge(alpha=1.0, random_state=RANDOM_STATE))])
    model_tf.fit(X_train_tf, y_train_tf)
    y_pred_tf = model_tf.predict(X_test_tf)
    mae_tf = mean_absolute_error(y_test_tf, y_pred_tf)
    print(f'Quick test (time features) 1-step Test MAE: {mae_tf:.2f} | rows train/test: {X_train_tf.shape}/{X_test_tf.shape}')
    print(f'Original 1-step Test MAE (no time feats): {mae_1:.2f}')
    delta = mae_1 - mae_tf
    print(f'MAE change (orig - time_feats) = {delta:.3f}  (positive => improvement)')


Quick test (time features) 1-step Test MAE: 97.62 | rows train/test: (3751, 109)/(903, 109)
Original 1-step Test MAE (no time feats): 129.85
MAE change (orig - time_feats) = 32.235  (positive => improvement)


/tmp/ipykernel_17011/4147035042.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat[f"lag_{lag}"] = y.shift(lag)
/tmp/ipykernel_17011/4147035042.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat[f"roll_mean_{w}"] = y.shift(1).rolling(w, min_periods=max(2, w//2)).mean()
/tmp/ipykernel_17011/4147035042.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

In [9]:
# Sign-only accuracy: compare sign(pred) == sign(true) for 1-step and rolling forecasts
import pandas as _pd
# 1-step test set accuracy (uses y_pred_1 and y_test from above)
try:
    pred_sign_1 = np.sign(y_pred_1)
    true_sign_1 = np.sign(y_test)
    sign_acc_1 = (pred_sign_1 == true_sign_1).mean() * 100
    print(f'1-step sign accuracy: {sign_acc_1:.2f}%   (n={len(true_sign_1)})')
except NameError:
    print('1-step variables y_pred_1 / y_test not found in scope; run the 1-step training cell first')

# Rolling multi-step sign accuracy per horizon (requires rolling_forecasts / valid)
if 'rolling_forecasts' in globals():
    rf = rolling_forecasts.copy()
    rf = rf.dropna(subset=['y_true'])
    if rf.empty:
        print('No valid rolling forecasts with y_true available to compute sign accuracy.')
    else:
        rf['pred_sign'] = np.sign(rf['y_pred'].values)
        rf['true_sign'] = np.sign(rf['y_true'].values)
        overall = (rf['pred_sign'] == rf['true_sign']).mean() * 100
        print(f'Rolling forecasts overall sign accuracy: {overall:.2f}%   (rows={len(rf)})')
        per_h = rf.groupby('h').apply(lambda g: _pd.Series({'SignAcc': (g['pred_sign']==g['true_sign']).mean()*100, 'Count': len(g)})).reset_index()
        print('Per-horizon sign accuracy (first 10 horizons):')
        print(per_h.head(10))
else:
    print('rolling_forecasts not found in scope; run the rolling forecast evaluation cell first')


1-step sign accuracy: 74.23%   (n=4312)
Rolling forecasts overall sign accuracy: 51.99%   (rows=38400)
Per-horizon sign accuracy (first 10 horizons):
    h  SignAcc  Count
0   1     75.0  200.0
1   2     73.0  200.0
2   3     67.5  200.0
3   4     65.5  200.0
4   5     56.0  200.0
5   6     56.0  200.0
6   7     56.5  200.0
7   8     64.5  200.0
8   9     52.0  200.0
9  10     57.0  200.0


In [10]:
# Save 1-step sign forecast results to CSV
# Columns: origin_ts,h,target_ts,y_true,y_pred,pred_state,correctness,correctness_label
from pathlib import Path
try:
    out_dir = Path("neoen_morcenx/outputs_same_tod")
    out_dir.mkdir(parents=True, exist_ok=True)

    # test_feat contains 'ts' and 'y_next' used earlier to build X_test / y_test
    origin_ts = test_feat['ts']
    df_1 = pd.DataFrame({
        'origin_ts': origin_ts,
        'h': 1,
        'target_ts': origin_ts + pd.Timedelta(minutes=STEP_MINUTES),
        'y_true': test_feat['y_next'].values,
        'y_pred': y_pred_1,
    })

    # predicted state as sign (-1, 0, 1)
    df_1['pred_state'] = np.sign(df_1['y_pred']).astype(int)
    # correctness: sign(pred) == sign(true)
    df_1['correctness'] = (np.sign(df_1['y_pred']) == np.sign(df_1['y_true']))
    df_1['correctness_label'] = df_1['correctness'].map({True: 'correct', False: 'incorrect'})

    out_path = out_dir / 'one_step_sign_forecasts.csv'
    df_1.to_csv(out_path, index=False)
    print(f"Saved 1-step sign forecasts to {out_path} | rows={len(df_1)}")
    try:
        display(df_1.head())
    except NameError:
        # display may not be available in some runtimes; silently continue
        pass
except Exception as e:
    print('Could not save 1-step sign forecasts:', e)

Saved 1-step sign forecasts to neoen_morcenx/outputs_same_tod/one_step_sign_forecasts.csv | rows=4312


,origin_ts,h,target_ts,y_true,y_pred,pred_state,correctness,correctness_label
0,2025-08-20 00:00:00+00:00,1,2025-08-20 00:15:00+00:00,-165.0,-220.668189,-1,True,correct
1,2025-08-20 00:15:00+00:00,1,2025-08-20 00:30:00+00:00,-343.0,-187.433576,-1,True,correct
2,2025-08-20 00:30:00+00:00,1,2025-08-20 00:45:00+00:00,-198.0,-70.617153,-1,True,correct
3,2025-08-20 00:45:00+00:00,1,2025-08-20 01:00:00+00:00,-49.0,-78.478030,-1,True,correct
4,2025-08-20 01:00:00+00:00,1,2025-08-20 01:15:00+00:00,-59.0,-50.345318,-1,True,correct
